# 日本語データセットをチャンクに分割

In [ ]:
#!pip install tiktoken langchain

In [ ]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

## 1. 単一ファイルの動作確認

In [ ]:
f = open('./data/源頼朝.txt', 'r', encoding='UTF-8')

data = f.read()
print(data)

f.close()

## ドキュメントのトークン数を計測
OpenAI モデルのトークン数を正確に数えるためには、[tiktoken](https://github.com/openai/tiktoken) ライブラリを利用します。

In [ ]:
enc = tiktoken.get_encoding("gpt2")
print(len(enc.encode(data)))

## テキストをチャンクに分割
通常 1 つのドキュメントに含まれるトークン数は 1 度のコンテキストに指定できる最大トークン数をゆうに超えます。今回はドキュメントの文を指定したトークン数以下のチャンクに分割します。

- テキストの分割方法<br>
RecursiveCharacterTextSplitter はチャンクが十分に小さくなるまで、順番に分割します。
- チャンクサイズの測定方法<br>
tiktoken ライブラリのトークナイザーを使用してトークン数を正確に測定します。

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='gpt2',
    chunk_size=500, 
    chunk_overlap=50
)

In [ ]:
chunk = text_splitter.split_text(data)
chunk

In [ ]:
#チャンク数
len(chunk)

In [ ]:
#チャンク[0]の中身
chunk[0]

In [ ]:
#トークン数
print(len(enc.encode(chunk[0])))

In [ ]:
#チャンクごとのトークン数確認
tokencounter = 0
for i, text in enumerate(chunk):
    print(i,len(enc.encode(text)))
    tokencounter = tokencounter + len(enc.encode(text))

print(tokencounter)

## 2. ファイル一括処理
ドキュメントをチャンクに分割する方法を確認したら、実際に複数のデータセットを一括処理でチャンクごとのテキストファイルに分割します。

In [ ]:
import os

enc = tiktoken.get_encoding("gpt2")
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='gpt2',
    chunk_size=500, 
    chunk_overlap=50
)

#テキストファイルを読み込んで、指定のトークン数のチャンクファイルに分割します。
def splitChunkFile(filepath):
    f = open(filepath, 'r', encoding='UTF-8')
    data = f.read()
    chunk = text_splitter.split_text(data)

    #chunk単位でループ
    for i, chunkedtext in enumerate(chunk):
        
        dirname = os.path.dirname(filepath)
        basename = os.path.splitext(os.path.basename(filepath))[0]
        outputfilepath = dirname + "/output/" + basename + "-" + str(i) + ".txt"
        
        print(i, len(enc.encode(chunkedtext)), outputfilepath)
        with open(outputfilepath, 'w', encoding='UTF-8') as fo:
            fo.write(chunkedtext)

        fo.close()
    f.close()
   
    return

In [ ]:
import glob
for p in glob.glob('./data/*.txt'):
    splitChunkFile(p)